Importar librerias requeridas

In [1]:
import ee 
import geemap
import json
import os

Autenticación con GEE y extracción de zona de estudio

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-rimartinezs')

with open(os.path.join(os.getcwd(), '..', 'data', 'Localidades_Bogota.geojson')) as geojson:
    data = json.load(geojson)

zona_estudio = ee.FeatureCollection(data)


Extracción de la colección e imagenes de GEE

In [6]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(zona_estudio) \
    .filterDate('2023-01-01', '2023-12-31')

print('Número de imágenes:', landsat8.size().getInfo())
image = landsat8.sort('CLOUD_COVER').first()
image_id = image.getInfo()['id']
print("Id imagen a trabajar",image_id)

Número de imágenes: 18
Id imagen a trabajar LANDSAT/LC08/C02/T1_L2/LC08_008057_20230603


Visualización y configuración de la imagen

In [4]:
style = {
    "color": "#154360",         
    "fillColor": "#d4e6f1", 
    "width": 2,    
}
rgb_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 7000,
    'max': 20000,
    'gamma': 2.5
}

reflectance_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.00,
    'max': 0.12,
    'gamma': 2.0
}
escala = 0.0000275
intercepto = -0.2

image_SR = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']) \
    .multiply(escala).add(intercepto)

image_clipped = image.clip(zona_estudio)
image_clipped_SR = image_SR.clip(zona_estudio)
# RGB 572
false_color_572 = {
    'bands': ['SR_B5', 'SR_B7', 'SR_B2'],
    'min': 0.00,
    'max': 0.35,
    'gamma': 1.5
}


# RGB 564
false_color_564 = {
    'bands': ['SR_B5', 'SR_B6', 'SR_B4'],
    'min': 0.00,
    'max': 0.35,
    'gamma': 1.5
}


Map = geemap.Map()
Map = geemap.Map(height=300, width='70%') 
Map.centerObject(zona_estudio, zoom =10)
Map.addLayer(image_clipped, rgb_vis, "True Color")
Map.addLayer(image_clipped_SR,reflectance_vis,"True color without reflectance")
Map.addLayer(image_clipped_SR, false_color_564, "False Color 564")
Map.addLayer(image_clipped_SR, false_color_572, "False Color 572")

Map

Map(center=[4.314612563522231, -74.1806972663266], controls=(WidgetControl(options=['position', 'transparent_b…

Guardar Mapa

In [5]:
Map.to_html(os.path.join(os.getcwd(), '..', 'HTML', 'Composiciones_Color.html'))
